In [3]:
import tensorflow as tf
import tensorflow_decision_forests as tfdf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt
import numpy as np
import pickle
print(1)

1


In [2]:
X_train = pickle.load(open('./processed_facial_data/X_train.pickle', 'rb'))
y_train = pickle.load(open('./processed_facial_data/y_train.pickle', 'rb'))
X_test = pickle.load(open('./processed_facial_data/X_test.pickle', 'rb'))
y_test = pickle.load(open('./processed_facial_data/y_test.pickle', 'rb'))

In [3]:
# Normalizing data
X_train = np.array(X_train)
y_train = np.array(y_train)
X_train = X_train/255.0

X_test = np.array(X_test)
y_test = np.array(y_test)
X_test = X_test/255.0

In [4]:
# Attempting to extract features from images through CNN t-SNE
# conv2d layers are indexed at [1, 6]
model = load_model('./facial_emotion_recognition.h5')

In [5]:
layer_names = [layer.name for layer in model.layers]
layer_outputs = [layer.output for layer in model.layers]

In [6]:
feature_map_model = Model(inputs = model.input, outputs=layer_outputs)

In [7]:
new_img = X_train[0].reshape(1, X_train[0].shape[0], X_train[0].shape[1])
features = feature_map_model.predict(new_img)

1/1 [==============================] - 3s 3s/step


In [8]:
# visualizing feature shapes
# for feature in features:
#     print(feature.shape)

In [9]:
# plt.imshow(X_test[0])
# y_test[0]

In [10]:
# visualizing the features from the cnn
# first conv2d layer has 32 filters
# second conv2d layer has 64 filters
# features are arranged as:
# features[cnn_layer_index][image_index,dimension1,dimension2,filter_index]
# hint: for filter_index, look at the amount of filters you are adding per conv2d layer (32, 64, etc)

# for i in range(len(layer_names)):
#     if layer_names[i].split('_')[0] == 'activation':
#         print(f'{i}: {layer_names[i]}')
#         try:
#             plt.imshow(features[i][0,:,:,15], cmap='Greys')
#             plt.show()
#         except:
#             print(features[i].shape)
#             continue

In [11]:
# attempting to flatten features from 'activation' layer
flattened = features[2][0,:,:,31].ravel()
# features[2][0,:,:,31].shape
# features

In [12]:
flattened.shape

(2116,)

In [13]:
# get length of features
img = X_train[0].reshape(1,X_train[0].shape[0],X_train[0].shape[1])
feature_map_model = Model(inputs = model.input, outputs=layer_outputs)
sample = feature_map_model.predict(img)

# # defining the training data matrix
# train_data = np.ndarray([X_train.shape[0],len(sample), 211], dtype=object)
# print(train_data.shape)
# print(type(train_data))
# print(train_data)
# print(train_data[0][26])

1/1 [==============================] - 0s 82ms/step


In [14]:
# features gets you a (1,dim_1,dim_2,num_filters) shaped data
# for the layers that are conv2d, we will arrannge the data as [[filter_1], [filter_2],...,[filter_n]]
# features_train[19][0]

In [15]:
# starting with the first two images of the dataset, then we can generalize onto the rest of the images
# X_train.shape[0]
for j in range(1):
    # adding an additional dimension to image (1,dim_1,dim_2)
    img = X_train[j].reshape(1, X_train[j].shape[0], X_train[j].shape[1])
    # extracting features from the images in training set
    features_train = feature_map_model.predict(img)
    print(f'FOR X_train[{j}]')
    for k in range(len(features_train)):
        conv_features = []
        dense_features = []
        if len(features_train[k].shape) == 4:       # extract features from conv2d layers
            for l in range(features_train[k].shape[3]):  # append all conv2d filters (32 or 64 filters) to our conv_features array
                conv_features.append( features_train[k][0,:,:,l].ravel() )

        elif len(features_train[k].shape) == 2:     # extract features from dense layers
            dense_features.append(features_train[k][0])
        
        
        if len(conv_features) != 0:
            conv_features = np.array(conv_features)
            print(f'conv_features length = {len(conv_features)} with conv_features.shape = {conv_features.shape}, total_parameters = {conv_features.shape[0]*conv_features.shape[1]}')
        
        if len(dense_features) != 0:
            dense_features = np.array(dense_features)
            print(f'dense_features length = {len(dense_features)} with dense_features.shape = {dense_features.shape}')
    print('')

1/1 [==============================] - 0s 15ms/step
FOR X_train[0]
conv_features length = 32 with conv_features.shape = (32, 2116), total_parameters = 67712
conv_features length = 32 with conv_features.shape = (32, 2116), total_parameters = 67712
conv_features length = 32 with conv_features.shape = (32, 2116), total_parameters = 67712
conv_features length = 32 with conv_features.shape = (32, 529), total_parameters = 16928
conv_features length = 32 with conv_features.shape = (32, 529), total_parameters = 16928
conv_features length = 64 with conv_features.shape = (64, 529), total_parameters = 33856
conv_features length = 64 with conv_features.shape = (64, 529), total_parameters = 33856
conv_features length = 64 with conv_features.shape = (64, 529), total_parameters = 33856
conv_features length = 64 with conv_features.shape = (64, 121), total_parameters = 7744
conv_features length = 64 with conv_features.shape = (64, 121), total_parameters = 7744
conv_features length = 64 with conv_featur

In [16]:
# concluding from the above printing, we can easily now picture how our dataset would look like. 
# We will have 27 different datasets, and we will store them into an array that will have the following shape:
# train_data = [conv_feature_1, conv_feature_2, ..., conv_feature_n, dense_feature_1, ..., dense_feature_m]
# where the features of the layers would look like this:
# conv_feature_x ----> length = num_filters
#                ----> array_entry = 1-D array of images
#                ----> each filter will have the same label 
# dense_feature_x ---> length = num_neurons

In [17]:
print(f'validation data length: {len(X_train)}')
print(f'training data length: {int(0.5*len(X_test))}')
print(f'testing data length: {int(0.5*len(X_test))}')

validation data length: 28709
training data length: 3589
testing data length: 3589


In [18]:
# creating training set
train_data = []
train_percent = -1
train_size = X_train.shape[0]

print('Generating training set...')

for j in range(train_size):
    #Calculate new progress percentage
    newPercent = int((j/train_size) * 100)
    if newPercent != train_percent:
        train_percent = newPercent
        print(f'{train_percent}% | {j}/{train_size}')

    # adding an additional dimension to image (1,dim_1,dim_2)
    img = X_train[j].reshape(1, X_train[j].shape[0], X_train[j].shape[1])
    # extracting features from the images in training set
    features_train = feature_map_model.predict(img,verbose=None)
    for k in range(len(features_train)):
        if len(features_train[k].shape) == 4:       # extract features from conv2d layers
            conv_features = []
            for l in range(features_train[k].shape[3]):  # append all conv2d filters (32 or 64 filters) to our conv_features array
                conv_features.append( features_train[k][0,:,:,l].ravel() )
            # flattening filter vectors
            conv_features = np.array(conv_features).ravel()
            # storing our feature vectors
            train_data.append( (conv_features, y_train[j], X_train[j]) )

        elif len(features_train[k].shape) == 2:     # extract features from dense layers
            dense_features = features_train[k][0]
            train_data.append( (dense_features, y_train[j], X_train[j]) )

print('Finished generating training set!')     

Generating training set...
0% | 0/28709
1% | 288/28709
2% | 575/28709
3% | 862/28709
4% | 1149/28709
5% | 1436/28709
6% | 1723/28709
7% | 2010/28709
8% | 2297/28709
9% | 2584/28709
10% | 2871/28709
11% | 3158/28709
12% | 3446/28709
13% | 3733/28709
14% | 4020/28709
15% | 4307/28709
16% | 4594/28709
17% | 4881/28709
18% | 5168/28709
19% | 5455/28709
20% | 5742/28709
21% | 6029/28709
22% | 6316/28709
23% | 6604/28709
24% | 6891/28709
25% | 7178/28709
26% | 7465/28709
27% | 7752/28709
28% | 8039/28709
29% | 8326/28709
30% | 8613/28709
31% | 8900/28709
32% | 9187/28709
33% | 9474/28709
34% | 9762/28709
35% | 10049/28709
36% | 10336/28709
37% | 10623/28709
38% | 10910/28709
39% | 11197/28709
40% | 11484/28709
41% | 11771/28709
42% | 12058/28709
43% | 12345/28709
44% | 12632/28709
45% | 12920/28709
46% | 13207/28709
47% | 13494/28709
48% | 13781/28709
49% | 14068/28709
50% | 14355/28709
51% | 14642/28709
52% | 14929/28709
53% | 15216/28709
54% | 15503/28709
55% | 15790/28709
56% | 16078/2870

In [19]:
# creating the testing and validation dataset (50/50 split)
testing_data = []
val_data = []
testing_percent = -1
testing_size = X_test.shape[0]
data_split = int(0.5*len(X_test))

print('Generating test set...')

for j in range(testing_size):
    #Calculate new progress percentage
    newPercent = int((j/testing_size) * 100)
    if newPercent != testing_percent:
        testing_percent = newPercent
        print(f'{testing_percent}% | {j}/{testing_size}')
        
    # adding an additional dimension to image (1,dim_1,dim_2)
    img = X_test[j].reshape(1, X_test[j].shape[0], X_test[j].shape[1])
    # extracting features from the images in training set
    features_test = feature_map_model.predict(img,verbose=None)
    for k in range(len(features_test)):
        if len(features_test[k].shape) == 4:       # extract features from conv2d layers
            conv_features = []
            for l in range(features_test[k].shape[3]):  # append all conv2d filters (32 or 64 filters) to our conv_features array
                conv_features.append( features_test[k][0,:,:,l].ravel() )
            # flattening filter vectors
            conv_features = np.array(conv_features).ravel()
            # storing our feature vectors
            if j < data_split:    #fill the test set
                testing_data.append( (conv_features, y_test[j], X_test[j]) )
            else:    # fill the validation set
                val_data.append( (conv_features, y_test[j], X_test[j]) )

        elif len(features_test[k].shape) == 2:     # extract features from dense layers
            dense_features = features_test[k][0]
            if j < data_split:    #fill the test set
                testing_data.append( (dense_features, y_test[j], X_test[j]) )
            else:    # fill the validation set
                val_data.append( (dense_features, y_test[j], X_test[j]) )
            
print('Finished generating test sets!')

Generating test set...
0% | 0/7178
1% | 72/7178
2% | 144/7178
3% | 216/7178
4% | 288/7178
5% | 359/7178
6% | 431/7178
7% | 503/7178
8% | 575/7178
9% | 647/7178
10% | 718/7178
11% | 790/7178
12% | 862/7178
13% | 934/7178
14% | 1005/7178
15% | 1077/7178
16% | 1149/7178
17% | 1221/7178
18% | 1293/7178
19% | 1364/7178
20% | 1436/7178
21% | 1508/7178
22% | 1580/7178
23% | 1651/7178
24% | 1723/7178
25% | 1795/7178
26% | 1867/7178
27% | 1939/7178
28% | 2010/7178
29% | 2082/7178
30% | 2154/7178
31% | 2226/7178
32% | 2297/7178
33% | 2369/7178
34% | 2441/7178
35% | 2513/7178
36% | 2585/7178
37% | 2656/7178
38% | 2728/7178
39% | 2800/7178
40% | 2872/7178
41% | 2943/7178
42% | 3015/7178
43% | 3087/7178
44% | 3159/7178
45% | 3231/7178
46% | 3302/7178
47% | 3374/7178
48% | 3446/7178
49% | 3518/7178
50% | 3589/7178
51% | 3661/7178
52% | 3733/7178
53% | 3805/7178
54% | 3877/7178
55% | 3948/7178
56% | 4020/7178
57% | 4092/7178
58% | 4164/7178
59% | 4236/7178
60% | 4307/7178
61% | 4379/7178
62% | 4451/7

In [20]:
train_data[0]

(array([-0.03288623, -0.01935673, -0.01263739, ...,  0.00850673,
         0.00559484,  0.00593166], dtype=float32),
 5,
 array([[0.70196078, 0.85490196, 0.92941176, ..., 0.65882353, 0.64705882,
         0.62745098],
        [0.69411765, 0.85882353, 0.94117647, ..., 0.66666667, 0.63921569,
         0.63137255],
        [0.69019608, 0.85098039, 0.9372549 , ..., 0.68235294, 0.64313725,
         0.63529412],
        ...,
        [0.56470588, 0.65490196, 0.69803922, ..., 0.12941176, 0.08627451,
         0.03921569],
        [0.56078431, 0.65098039, 0.69411765, ..., 0.10980392, 0.06666667,
         0.03921569],
        [0.56078431, 0.64705882, 0.69411765, ..., 0.07843137, 0.04705882,
         0.04705882]]))

In [21]:
# for i in range(0,1000,27):
#     print(f'{i}.) train_data_shape = {train_data[i][0].shape}, label = {train_data[i][1]}', end='\n\n')
#     print(train_data[i][1],end='\n\n')


In [22]:
# WARNING: DO NOT run this cell unless you want to kill your kernel!!! Data is too big (7GB) :)
# np_train_data = np.array(train_data)
# np.save('GBD_train_data',np_train_data)

In [23]:
def create_datasets(data, j, layer_names, dtype):
    # next create a function that separates the data into datasets per layer
    # input arguments are:
        # --> data = an array of tuples arranged as (data, label)
        # --> j = index to keep track of the layer name
        # --> layer_names = array with the names of layers of our CNNs
        # --> dtype = string with type of data we want to create (train, test, validation)
    # output will be the data saved as .npy files in their respective folders
    
    # only save layers
    if j < 20:
        return
    
    # i will keep track of the indices of train_data and be updated by +27
    i = j
    output_data = []
    output_labels = []
    output_imgs = []
    
    while i < len(data):
        # will try to save dataset with layer_name files
        output_data.append( data[i][0] )
        output_labels.append( data[i][1] )
        output_imgs.append( data[i][2] )
        i+=27
    file_data = f'./GBT_data/{dtype}/data/{layer_names[j]}_layer_{j}_data'
    file_labels = f'./GBT_data/{dtype}/labels/{layer_names[j]}_layer_{j}_labels'
    np.save(file_data, output_data)
    np.save(file_labels, output_labels)
    
    if(j == 26):
        file_imgs = f'./GBT_data/{dtype}/labels/img'
        np.save(file_imgs, output_imgs)

In [24]:
for j in range(len(layer_names)):
    create_datasets(train_data, j, layer_names, dtype='train')
    create_datasets(val_data, j, layer_names, dtype='validation')
    create_datasets(testing_data, j, layer_names, dtype='test')

In [27]:
len(train_data)/27

20096.0